In [1]:
# Custom libraries
from datascienceutils import clusteringModels as cm
from datascienceutils import analyze
from datascienceutils import plotter
from datascienceutils import sklearnUtils as sku

# Standard libraries
import json
%matplotlib inline
import datetime
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file, output_notebook, ColumnDataSource
from bokeh.charts import Histogram
import bokeh
output_notebook()


/home/anand/playspace/data-science-utils/.eggs/statsmodels-0.8.0-py3.6-linux-x86_64.egg/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/anand/anaconda3/envs/analytics/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


SyntaxError: keyword can't be an expression (statsutils.py, line 62)

In [ ]:
from sklearn import datasets
# Load the diabetes dataset
diabetes = datasets.load_diabetes()
# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

In [ ]:
CHECK_DISTS = ['norm', 'expon', 'logistic', 'cosine', 'cauchy',]

def distribution_tests(series, test_type='ks', dist_type=None):
    from scipy import stats
    if not dist_type:
        ks_test_results = pd.DataFrame(columns=['distribution', 'statistic', 'p-value'])
        for distribution in CHECK_DISTS:
            if test_type=='ks':
                print("Kolmogrov - Smirnov test with distribution %s"%distribution)
                try:
                    stat, pval = stats.kstest(series, distribution)
                except Exception:
                    print("Error for dist: %s"%distribution)
                    continue
                print(stat, pval)
                ks_test_results.append({'distribution': distribution,
                                        'statistic':stat,
                                        'p-value':pval},ignore_index=True)
            elif test_type =='wald':
                print("Wald test with distribution %s"%distribution)
                print(lm.wald_test(series, distribution))
            else:
                raise "Unknown distribution similarity test type"
        print(ks_test_results)
    else:
        print(stats.kstest(series, dist_type))
distribution_tests(diabetes_X[0])

In [ ]:
from datascienceutils import statsutils as su
su.distribution_tests(diabetes_X[0],)

In [ ]:
plotter.show(plotter.lineplot(pd.DataFrame(diabetes_X[0], columns=['0'])))

In [ ]:
from sklearn.externals import joblib

import os
import glob
files = glob.glob('../models/*.pkl')

print(files)

In [ ]:
from datascienceutils import settings
settings.MODELS_BASE_PATH='../models'
models = dict()
for i,file in enumerate(files):
    model = file.split('/')[-1][:-4]
    models[i] = sku.load_model(model)
    

In [ ]:
models.items()

In [ ]:
res = list()
input_data = diabetes_X_test
for k,(model, metadata) in models.items():
       
    res.append(model.predict(diabetes_X_test))
    model_info = metadata
    model_info.update({'model_class': 'regression'})
    model_info.update({'output_metadata': {'dist':'norm'}})    # Declare predictions to belong to normal distribution
    if model_info['model_class'] == 'regression':
        predictions = model.predict(input_data)
        # Check the predictions are type of continuous variables (float or int)
        # parse and translate output_metadata to choice of tests
        dist = model_info['output_metadata']['dist']
        print(su.distribution_similarity(predictions, dist_type=dist))

    


In [ ]:
print(res)